In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/features/SelectedFeaturesVGGFinalOS.csv
/kaggle/input/features/FeatureStdVGGFinalTESTOS.csv
/kaggle/input/features/FeatureStdAlexFinalTESTOS.csv
/kaggle/input/features/FeatureAvgVGGFinal.csv
/kaggle/input/features/SelectedFeaturesVGGFinalMergedOS.csv
/kaggle/input/features/FeatureAvgVGGFinalTESTOS.csv
/kaggle/input/features/SelectedFeaturesVGGFinal.csv
/kaggle/input/features/FeatureAvgAlexFinal.csv
/kaggle/input/features/SelectedFeaturesVGGFinalMerged.csv
/kaggle/input/features/SelectedFeaturesVGGFinalMergedTESTOS.csv
/kaggle/input/features/SelectedFeaturesAlexFinalMerged.csv
/kaggle/input/features/SelectedFeaturesVGGFinalTESTOS.csv
/kaggle/input/features/FeatureStdVGGFinal.csv
/kaggle/input/features/FeatureStdVGGFinalOS.csv
/kaggle/input/features/SelectedFeaturesAlexFinalTESTOS.csv
/kaggle/input/features/SelectedFeaturesAlexFinal.csv
/kaggle/input/features/FeatureTargetVGGFinal.csv
/kaggle/input/features/FeatureAvgVGGTESTOS.csv
/kaggle/input/features/FeatureTargetAlexFin

In [3]:
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn import tree
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
tf.random.set_seed(42)

In [4]:
x_avg = pd.read_csv("/kaggle/input/features/FeatureAvgAlexFinalOS.csv")
x_std = pd.read_csv("/kaggle/input/features/FeatureStdAlexFinalOS.csv")
y_train = pd.read_csv("/kaggle/input/features/FeatureTargetAlexFinalOS.csv")
y = pd.read_csv("/kaggle/input/features/FeatureTargetAlexFinalOS.csv")
y = y.iloc[:, [1]]
print(y['Survival_binary'].dtype)

test_df = pd.read_csv("/kaggle/input/features/test_df.csv")


y_train = y_train.iloc[:, [1]]

x_avg_test = pd.read_csv("/kaggle/input/features/FeatureAvgAlexFinalTESTOS.csv")
x_std_test = pd.read_csv("/kaggle/input/features/FeatureStdAlexFinalTESTOS.csv")
y_train_test = pd.read_csv("/kaggle/input/features/FeatureTargetAlexFinalTESTOS.csv")


y_train_test = y_train_test.iloc[:, [1]]



int64


In [5]:
x_avg.head()

,AlexNet_Avg_F0,AlexNet_Avg_F1,AlexNet_Avg_F2,AlexNet_Avg_F3,AlexNet_Avg_F4,AlexNet_Avg_F5,AlexNet_Avg_F6,AlexNet_Avg_F7,AlexNet_Avg_F8,AlexNet_Avg_F9,...,AlexNet_Avg_F246,AlexNet_Avg_F247,AlexNet_Avg_F248,AlexNet_Avg_F249,AlexNet_Avg_F250,AlexNet_Avg_F251,AlexNet_Avg_F252,AlexNet_Avg_F253,AlexNet_Avg_F254,AlexNet_Avg_F255
0,0.000374,0.042427,0.066797,0.007308,0.000046,0.084987,1.259873,0.031055,0.000751,0.096842,...,0.000917,0.002009,0.400057,0.054643,0.105998,0.940495,0.042096,0.592592,0.076810,0.042103
1,0.001945,0.041329,0.078117,0.000885,0.000154,0.099475,1.353625,0.010629,0.006738,0.250591,...,0.001235,0.001135,0.955365,0.024682,0.072329,0.359375,0.081209,0.578653,0.062131,0.073096
2,0.002961,0.036976,0.144324,0.006022,0.006292,0.223103,2.224358,0.015679,0.035390,0.051285,...,0.000637,0.009502,0.534150,0.076820,0.116177,0.732095,0.068831,0.624203,0.119198,0.049052
3,0.002229,0.023280,0.039875,0.020544,0.000184,0.067871,1.726983,0.029223,0.007647,0.078217,...,0.002807,0.021293,0.586790,0.019140,0.073723,0.516912,0.029411,0.625935,0.074210,0.040071
4,0.001826,0.022827,0.017068,0.002691,0.001559,0.107797,1.147238,0.028763,0.000003,0.147898,...,0.000882,0.006140,0.636418,0.057201,0.005395,0.262437,0.063906,0.750830,0.029361,0.033264


In [6]:
x_std.head()

,AlexNet_Std_F0,AlexNet_Std_F1,AlexNet_Std_F2,AlexNet_Std_F3,AlexNet_Std_F4,AlexNet_Std_F5,AlexNet_Std_F6,AlexNet_Std_F7,AlexNet_Std_F8,AlexNet_Std_F9,...,AlexNet_Std_F246,AlexNet_Std_F247,AlexNet_Std_F248,AlexNet_Std_F249,AlexNet_Std_F250,AlexNet_Std_F251,AlexNet_Std_F252,AlexNet_Std_F253,AlexNet_Std_F254,AlexNet_Std_F255
0,0.002128,0.060166,0.094494,0.013708,0.000397,0.042453,0.361317,0.047386,0.002522,0.092039,...,0.003381,0.004940,0.259386,0.082711,0.057867,0.270276,0.037434,0.147936,0.086578,0.070345
1,0.008416,0.059405,0.109643,0.006171,0.000942,0.067850,0.385692,0.032134,0.015041,0.118155,...,0.006092,0.006788,0.667349,0.028850,0.060325,0.204344,0.084894,0.182870,0.076749,0.128031
2,0.009969,0.047950,0.149918,0.014066,0.010206,0.181994,0.253982,0.017966,0.049651,0.059949,...,0.003700,0.016842,0.260538,0.071717,0.110564,0.425334,0.075467,0.153674,0.153151,0.079805
3,0.010240,0.030216,0.065348,0.034549,0.001048,0.053437,0.371911,0.043961,0.015041,0.068596,...,0.010356,0.020193,0.282370,0.020016,0.070781,0.264353,0.041177,0.204733,0.107178,0.048623
4,0.005900,0.029142,0.052758,0.004949,0.004888,0.100951,0.263424,0.038265,0.000020,0.144775,...,0.003878,0.009201,0.435877,0.049857,0.009594,0.170952,0.040667,0.260383,0.059850,0.038089


In [7]:
from sklearn.preprocessing import StandardScaler
scaler_avg = StandardScaler()
scaler_std = StandardScaler()
scaler_avg_test = StandardScaler()
scaler_std_test = StandardScaler()

scaler_avg.fit(x_avg.values)
scaler_std.fit(x_std.values)

scaler_avg_test.fit(x_avg_test.values)
scaler_std_test.fit(x_std_test.values)

StandardScaler()

In [8]:
len(scaler_avg.mean_)

256

In [9]:
# scaler.transform(x_train)
x_avg = pd.DataFrame(scaler_avg.transform(x_avg.values), columns=x_avg.columns, index=x_avg.index)
x_std = pd.DataFrame(scaler_std.transform(x_std.values), columns=x_std.columns, index=x_std.index)

x_avg_test = pd.DataFrame(scaler_avg_test.transform(x_avg_test.values), columns=x_avg_test.columns, index=x_avg_test.index)
x_std_test = pd.DataFrame(scaler_std_test.transform(x_std_test.values), columns=x_std_test.columns, index=x_std_test.index)

x_avg.head()

,AlexNet_Avg_F0,AlexNet_Avg_F1,AlexNet_Avg_F2,AlexNet_Avg_F3,AlexNet_Avg_F4,AlexNet_Avg_F5,AlexNet_Avg_F6,AlexNet_Avg_F7,AlexNet_Avg_F8,AlexNet_Avg_F9,...,AlexNet_Avg_F246,AlexNet_Avg_F247,AlexNet_Avg_F248,AlexNet_Avg_F249,AlexNet_Avg_F250,AlexNet_Avg_F251,AlexNet_Avg_F252,AlexNet_Avg_F253,AlexNet_Avg_F254,AlexNet_Avg_F255
0,-0.454000,-0.230860,0.137106,-0.448268,-0.383218,0.098718,0.451090,1.014045,-0.760153,-0.551023,...,-0.466738,-0.538662,-0.553123,-0.732165,1.085499,3.957331,-0.381331,0.017719,0.074146,-0.105789
1,-0.034004,-0.253034,0.359772,-0.795182,-0.351884,0.349578,0.697259,-0.186531,-0.195385,0.240394,...,-0.424767,-0.635401,1.267340,-1.291878,0.392188,0.494354,0.530243,-0.047625,-0.190926,0.512620
2,0.237575,-0.340875,1.662070,-0.517723,1.443260,2.490196,2.983567,0.110323,2.507372,-0.785529,...,-0.503645,0.290303,-0.113526,-0.317874,1.295093,2.715446,0.241753,0.165911,0.839588,0.032864
3,0.042016,-0.617266,-0.392451,0.266700,-0.343064,-0.197647,1.677595,0.906423,-0.109706,-0.646897,...,-0.217270,1.594655,0.059043,-1.395404,0.420896,1.433144,-0.676982,0.174030,0.027187,-0.146343
4,-0.065692,-0.626424,-0.841074,-0.697612,0.059214,0.493681,0.155343,0.879378,-0.830755,-0.288218,...,-0.471295,-0.081685,0.221737,-0.684369,-0.986075,-0.083308,0.126970,0.759535,-0.782681,-0.282165


In [10]:
x_std.head()

,AlexNet_Std_F0,AlexNet_Std_F1,AlexNet_Std_F2,AlexNet_Std_F3,AlexNet_Std_F4,AlexNet_Std_F5,AlexNet_Std_F6,AlexNet_Std_F7,AlexNet_Std_F8,AlexNet_Std_F9,...,AlexNet_Std_F246,AlexNet_Std_F247,AlexNet_Std_F248,AlexNet_Std_F249,AlexNet_Std_F250,AlexNet_Std_F251,AlexNet_Std_F252,AlexNet_Std_F253,AlexNet_Std_F254,AlexNet_Std_F255
0,-0.444693,0.255770,0.833118,-0.260932,-0.497431,-0.389095,1.082699,1.527090,-0.977587,-0.137129,...,-0.519961,-0.622030,0.107081,1.201201,0.719765,1.338294,-0.379619,-0.555503,0.376345,0.409671
1,0.426953,0.232367,1.214500,-0.758980,-0.390603,0.266940,1.368716,0.706573,0.109306,0.276647,...,-0.276371,-0.438989,3.513845,-1.012400,0.813587,0.557395,1.191488,0.077160,0.150569,1.785368
2,0.642127,-0.120069,2.228477,-0.237269,1.424860,3.215457,-0.176784,-0.055615,3.114311,-0.645575,...,-0.491272,0.556564,0.116700,0.749354,2.731747,3.174790,0.879429,-0.451589,1.905600,0.635267
3,0.679728,-0.665647,0.099323,1.116386,-0.369824,-0.105354,1.207016,1.342814,0.109326,-0.508568,...,0.106795,0.888383,0.299015,-1.375477,1.212803,1.268137,-0.255695,0.473118,0.849552,-0.108354
4,0.078104,-0.698706,-0.217656,-0.839780,0.382576,1.121992,-0.065996,1.036415,-1.194798,0.698430,...,-0.475296,-0.200069,1.580899,-0.149069,-1.123315,0.161897,-0.272602,1.480946,-0.237614,-0.359575


In [11]:
x_train = x_avg.join(x_std)
x_train_test = x_avg_test.join(x_std_test)

In [12]:
x_train.shape,y_train.shape

((1596, 512), (1596, 1))

In [13]:
corr_matrix = x_train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
x_train.drop(to_drop, axis=1, inplace=True)
x_train_test.drop(to_drop, axis=1, inplace=True)

x_train.shape,x_train_test.shape

((1596, 342), (200, 342))

In [14]:
X_trains_df=pd.DataFrame(x_train,columns=x_train.columns)

In [15]:
y_train.head()

,Survival_binary
0,0
1,0
2,0
3,0
4,1


In [16]:
print(X_trains_df.isnull().sum())
print(X_trains_df.isin([np.nan, np.inf, -np.inf]).sum().sum())

print(pd.isnull(y_train).sum())
print(y_train.isin([np.nan, np.inf, -np.inf]).sum().sum())

AlexNet_Avg_F0      0
AlexNet_Avg_F1      0
AlexNet_Avg_F2      0
AlexNet_Avg_F3      0
AlexNet_Avg_F4      0
                   ..
AlexNet_Std_F240    0
AlexNet_Std_F248    0
AlexNet_Std_F249    0
AlexNet_Std_F251    0
AlexNet_Std_F253    0
Length: 342, dtype: int64
0
Survival_binary    0
dtype: int64
0


In [17]:
X_trains_df = X_trains_df.fillna(X_trains_df.mean())

In [18]:
print(X_trains_df.isnull().sum())
print(X_trains_df.isin([np.nan, np.inf, -np.inf]).sum().sum())

print(pd.isnull(y_train).sum())
print(y_train.isin([np.nan, np.inf, -np.inf]).sum().sum())

AlexNet_Avg_F0      0
AlexNet_Avg_F1      0
AlexNet_Avg_F2      0
AlexNet_Avg_F3      0
AlexNet_Avg_F4      0
                   ..
AlexNet_Std_F240    0
AlexNet_Std_F248    0
AlexNet_Std_F249    0
AlexNet_Std_F251    0
AlexNet_Std_F253    0
Length: 342, dtype: int64
0
Survival_binary    0
dtype: int64
0


## **SVM Feature Selection**

In [19]:
# dtc = tree.DecisionTreeClassifier()

svm = SVC(kernel="linear")


In [20]:
svm_selector_model = RFE(estimator=svm,n_features_to_select=120)
svm_selector_model_fit=svm_selector_model.fit(X_trains_df,y_train.values.ravel())


In [21]:
feat_index = pd.Series(data = svm_selector_model_fit.ranking_, index = x_train.columns)
signi_feat_rfe = feat_index[feat_index==1].index
print('Significant features from RFE using SVM',signi_feat_rfe)

Significant features from RFE using SVM Index(['AlexNet_Avg_F1', 'AlexNet_Avg_F3', 'AlexNet_Avg_F6', 'AlexNet_Avg_F8',
       'AlexNet_Avg_F11', 'AlexNet_Avg_F12', 'AlexNet_Avg_F18',
       'AlexNet_Avg_F19', 'AlexNet_Avg_F22', 'AlexNet_Avg_F23',
       ...
       'AlexNet_Std_F88', 'AlexNet_Std_F100', 'AlexNet_Std_F126',
       'AlexNet_Std_F129', 'AlexNet_Std_F132', 'AlexNet_Std_F143',
       'AlexNet_Std_F152', 'AlexNet_Std_F156', 'AlexNet_Std_F183',
       'AlexNet_Std_F237'],
      dtype='object', length=120)


In [22]:
print('Original number of features present in the dataset : {}'.format(x_train.shape[1]))
print('Number of features selected by SVM using Recursive feature selection is : {}'.format(len(signi_feat_rfe)))

Original number of features present in the dataset : 342
Number of features selected by SVM using Recursive feature selection is : 120


In [23]:
df_selected_features = x_train.filter(items=signi_feat_rfe, axis=1)
df_selected_features_test = x_train_test.filter(items=signi_feat_rfe, axis=1)

In [24]:
df_selected_features.head()

,AlexNet_Avg_F1,AlexNet_Avg_F3,AlexNet_Avg_F6,AlexNet_Avg_F8,AlexNet_Avg_F11,AlexNet_Avg_F12,AlexNet_Avg_F18,AlexNet_Avg_F19,AlexNet_Avg_F22,AlexNet_Avg_F23,...,AlexNet_Std_F88,AlexNet_Std_F100,AlexNet_Std_F126,AlexNet_Std_F129,AlexNet_Std_F132,AlexNet_Std_F143,AlexNet_Std_F152,AlexNet_Std_F156,AlexNet_Std_F183,AlexNet_Std_F237
0,-0.230860,-0.448268,0.451090,-0.760153,-0.617392,0.145388,-0.713352,3.072294,-3.343893,0.287640,...,-0.395601,-0.518436,1.409486,1.172180,0.300286,1.664165,0.429515,2.372739,0.392264,0.039178
1,-0.253034,-0.795182,0.697259,-0.195385,-0.166722,-0.300179,-0.610645,1.006501,-1.709687,0.445418,...,-0.273571,-0.730792,2.158627,1.716598,3.220307,1.408198,-0.042710,-0.806777,0.474495,-0.647180
2,-0.340875,-0.517723,2.983567,2.507372,-0.315714,1.890868,-0.603130,0.804748,-0.773580,0.722606,...,0.586920,3.010638,2.156915,0.932539,1.039452,2.013338,0.366237,1.555614,0.634674,0.464683
3,-0.617266,0.266700,1.677595,-0.109706,-0.592379,-0.929081,-0.664520,1.748768,-1.306905,0.013670,...,-0.270282,-0.054897,1.176164,1.144551,0.431733,1.598507,0.366779,0.675432,-0.155681,-0.532264
4,-0.626424,-0.697612,0.155343,-0.830755,0.427962,-0.564038,-0.113874,0.836717,-0.042087,0.700451,...,-0.016216,0.019415,0.295968,0.456092,1.261358,0.109802,-0.937823,-0.785477,-0.283774,-0.586898


In [25]:
df_selected_features.to_csv('SelectedFeaturesVGGFinalOS.csv',index = None,header = df_selected_features.columns)
df_selected_features_test.to_csv('SelectedFeaturesVGGFinalTESTOS.csv',index = None,header = df_selected_features_test.columns)



In [26]:
# skip NN 21 March
columns = ['VGG_F' + str(i) for i in range(df_selected_features.shape[1])]
df_selected_features.to_csv('SelectedFeaturesVGGFinalMergedOS.csv',index = None,header = columns)

columns = ['VGG_F' + str(i) for i in range(df_selected_features_test.shape[1])]
df_selected_features_test.to_csv('SelectedFeaturesVGGFinalMergedTESTOS.csv',index = None,header = columns)

In [27]:
df_selected_features_test.shape

(200, 120)

In [ ]:
# Step 6: Train an SVM model using Stratified Kfold cross validation and grid search cv
scaler = StandardScaler()
svc = SVC(probability=True)
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

mean_fpr = np.linspace(0, 1, 100)
tprs = []
aucs = []
best_svc = None
best_score = 0

for i, (train, test) in enumerate(cv.split(df_selected_features, y)):
    X_train = df_selected_features.iloc[train]
    y_train = y.iloc[train].values.ravel()
    X_test = df_selected_features.iloc[test]
    y_test = y.iloc[test].values.ravel()
    
#     common_indices = set(train).intersection(set(test))
#     print('Common indices:', common_indices if common_indices else None)

    grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=cv, scoring='roc_auc')
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    grid_search.fit(X_train_scaled, y_train)
    best_svc_fold = grid_search.best_estimator_
    
    score = grid_search.best_score_
    if score > best_score:
        best_score = score
        best_svc = grid_search.best_estimator_
    
    y_score = best_svc_fold.predict_proba(X_test_scaled)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_score)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    print('Fold %d ROC AUC: %.4f' % (i+1, roc_auc))


plt.figure(figsize=(8,6))
for i in range(len(tprs)):
    plt.plot(mean_fpr, tprs[i], lw=2, alpha=0.6, label='ROC fold %d (AUC = %0.2f)' % (i+1, aucs[i]))

plt.plot([0,1],[0,1],'--',lw=2, color='black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=2, alpha=.8)

# plt.fill_between(mean_fpr, mean_tpr - std_auc, mean_tpr + std_auc, alpha=.2, color='b')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.savefig('auc_roc.png')
plt.show()


In [ ]:
from sklearn.metrics import classification_report, plot_roc_curve

# Predict on test data
y_pred = best_svc.predict(df_selected_features_test)

# Print classification report
print(classification_report(y_train_test, y_pred))